In [23]:
import numpy as np
def average_precision_at_k(theta, labels, k, dist_fn):
    dist = dist_fn(theta)
    np.fill_diagonal(dist, np.inf)
    idx = np.argpartition(dist, k - 1)[:k, :]
    return np.mean(labels[idx] == labels[None, :])

In [24]:
from sklearn.metrics.pairwise import cosine_distances, euclidean_distances

def hellinger(theta):
    return euclidean_distances(np.sqrt(theta))

def dot(theta):
    return 1 - np.dot(theta, theta.transpose())

In [25]:
from numba import jit, prange, autojit

In [26]:
@jit(nopython = True, parallel = True)
def outer(X, fn):
    dist = np.empty((X.shape[0], X.shape[0]), dtype = np.float32)
    for i in prange(X.shape[0]):
        for j in prange(i):
            dist[j,i] = dist[i,j] = fn(X[i,:], X[j,:])           
    return dist

In [27]:
@jit(nopython = True)
def _dot(x,y):
    return np.dot(x,y)

In [28]:
@jit(nopython = True)
def kl(p, q):
    return np.dot(p, np.log(q / p))

In [29]:
@jit(nopython = True)
def sym_kl(p, q):
    return kl(p, q) + kl(q, p)

In [30]:
@jit(nopython = True)
def js(p, q):
    m = (p + q) / 2
    return kl(p, m) / 2 + kl(q, m) / 2

In [31]:
@jit(nopython = True)
def overlap(p, q):
    return 1 - np.sum(np.minimum(p,q))

In [32]:
@jit(nopython = True)
def l1_norm(p, q):
    return np.sum(np.abs(p - q))

In [33]:
@jit(nopython = True)
def bc(p, q):
    return -np.log(np.sum(np.sqrt(p * q)))

In [34]:
dist_fns = {'cosine' : cosine_distances,
            'eucl' : euclidean_distances,
            'hellinger' : hellinger,
            'dot' : dot,
            'sym_kl' : lambda x: outer(x, sym_kl),
            'js' : lambda x: outer(x, js),
            'overlap': lambda x: outer(x, overlap),
            'l1' : lambda x: outer(x, l1_norm),
            'bc' : lambda x: outer(x, bc)}

In [61]:
def eval_topic_model(corpus, model, labels, k = 100, dist_fns = dist_fns):
    theta = model.get_document_topics(corpus, minimum_probability = 0)
    theta = matutils.corpus2dense(theta, num_docs=len(corpus), num_terms=model.num_topics, dtype = np.float32)
    theta = np.asarray(theta, dtype = np.float32)
    theta = theta.transpose()
    best_score = 0
    best_dist = None
    for (dist, f) in dist_fns.items():
        score = average_precision_at_k(theta, labels, k, f)
        if score > best_score:
            best_score, best_dist = score, dist
    return best_score, best_dist

In [36]:
from sklearn.datasets import fetch_20newsgroups
cats = ['rec.autos', 'rec.motorcycles',
        'sci.crypt', 'sci.electronics', 
        'sci.med', 'sci.space',
        'talk.politics.guns', 'talk.religion.misc',
        'rec.sport.baseball', 'rec.sport.hockey']
newsgroups = fetch_20newsgroups(subset = 'all', categories=cats)

In [69]:
import re
import spacy
model = spacy.load('en', disable = ['parser','ner'])

def tokenize(string):
    email_pattern = re.compile(r'[^@]+@[^@]+\.[^@]+', re.IGNORECASE & re.UNICODE)
    header_pattern = re.compile(
        r'^summary:|^x-newsreader:.*|^date:.*'
        r'|^disclaimer:.*|^distribution:.*|^organization:.*'
        r'|^nntp-posting-host:.*|^keywords:.*|^to:.*'
        r'|^in-reply-to:.*|^x-news-reader:.*|^lines:.*',
        re.IGNORECASE & re.UNICODE)
    special_char_pattern = re.compile(r"[^\w']|_")
    number_pattern = re.compile(r'\b\d+\b')
    writes_pattern = re.compile(r'^.*writes:$')
    
    string = header_pattern.sub(' ', string)
    string = email_pattern.sub(' ', string)
    string = number_pattern.sub(' ', string)
    string = writes_pattern.sub(' ', string)
    string = special_char_pattern.sub(' ', string)
    
    string = ' '.join([w for w in string.split() if len(w) > 2])
    
    return [w.lemma_ for w in model(string) if w.pos_ not in {'SPACE','PUNCT'}]

In [70]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim
from gensim import matutils

en_stop = get_stop_words('en')
en_stop += ['would', 'subject', 're', 'don', 'jan','feb', 'mar', 'apr', 'may', 'june','july', 'aug', 'sep', 'oct', 'nov', 'dec']


p_stemmer = PorterStemmer()
texts = []
for i in newsgroups['data']:
    
    raw = i.lower()
    tokens = tokenize(raw)
    stopped_tokens = [i for i in tokens if not i in en_stop]
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    texts.append(stemmed_tokens)

In [71]:
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [72]:
from gensim import matutils
for num_topics in range(5, 61,5):
    model = gensim.models.ldamodel.LdaModel(corpus, num_topics=num_topics, id2word = dictionary, passes=20)
    score, dist = eval_topic_model(corpus, model, labels)
    print("%2i topics AP@100 = %.3f (%s distance)" % (num_topics, score, dist))

 5 topics AP@100 = 0.576 (sym_kl distance)
10 topics AP@100 = 0.698 (js distance)
15 topics AP@100 = 0.657 (js distance)
20 topics AP@100 = 0.632 (sym_kl distance)
25 topics AP@100 = 0.614 (js distance)
30 topics AP@100 = 0.569 (sym_kl distance)
35 topics AP@100 = 0.655 (js distance)
40 topics AP@100 = 0.615 (js distance)
45 topics AP@100 = 0.599 (sym_kl distance)
50 topics AP@100 = 0.592 (sym_kl distance)
55 topics AP@100 = 0.625 (sym_kl distance)
60 topics AP@100 = 0.626 (sym_kl distance)


In [81]:
model = gensim.models.ldamodel.LdaModel(corpus, num_topics=10, id2word = dictionary, passes=20)
for i in range(model.num_topics):
    print(model.print_topic(i, topn = 10))

0.020*"space" + 0.016*"nasa" + 0.012*"never" + 0.011*"gov" + 0.010*"com" + 0.009*"orbit" + 0.007*"satellit" + 0.007*"mission" + 0.007*"-pron-" + 0.006*"earth"
0.021*"edu" + 0.019*"com" + 0.011*"-pron-" + 0.011*"harvard" + 0.009*"ibm" + 0.007*"disclaim" + 0.007*"spdcc" + 0.007*"time" + 0.007*"berkeley" + 0.006*"ramsey"
0.018*"edu" + 0.016*"com" + 0.011*"chip" + 0.011*"new" + 0.010*"phone" + 0.009*"ripem" + 0.009*"net" + 0.008*"line" + 0.008*"appl" + 0.008*"use"
0.082*"-pron-" + 0.009*"can" + 0.009*"will" + 0.008*"one" + 0.007*"peopl" + 0.007*"write" + 0.006*"use" + 0.006*"say" + 0.006*"know" + 0.006*"get"
0.057*"-pron-" + 0.009*"get" + 0.008*"can" + 0.008*"will" + 0.008*"edu" + 0.008*"good" + 0.007*"'s" + 0.007*"one" + 0.006*"year" + 0.006*"just"
0.098*"com" + 0.017*"uunet" + 0.013*"netcom" + 0.011*"univers" + 0.011*"dod" + 0.010*"pat" + 0.010*"uucp" + 0.009*"seattl" + 0.008*"edu" + 0.008*"wisdom"
0.105*"edu" + 0.012*"gatech" + 0.010*"prism" + 0.008*"uc" + 0.006*"mil" + 0.006*"cray" + 0